In [11]:
from glob import glob
import uproot as ur
import numpy as np
import json
import os.path as osp
from phc import module_reload
module_reload('zhh')
from zhh import parse_sample_path, get_preselection_passes, ProcessIndex
version = 'v1'

REPO_ROOT = '/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH'
DATA_ROOT = f'/nfs/dust/ilc/user/bliewert/zhh/PreselectionFinal/{version}'

INDEX_DIR = '/nfs/dust/ilc/user/bliewert/zhh/CreateRawIndex/v1'
DATA_ROOT = '/nfs/dust/ilc/user/bliewert/zhh/PreselectionRuntime/v1'

processes = np.load(f'{INDEX_DIR}/processes.npy')
samples = np.load(f'{INDEX_DIR}/samples.npy')

Introduce indices on samples and processes (included in future runs)

In [12]:
dtype_sample = ProcessIndex.dtype_sample
dtype_process = ProcessIndex.dtype_process

if not 'sid' in samples.dtype.names:
    samples_new = np.empty(len(samples), dtype=dtype_sample)
    samples_new['sid'] = np.arange(len(samples))
    for col in samples.dtype.names:
        samples_new[col] = samples[col]
        
    if len(samples) == len(samples_new):
        np.save(f'{INDEX_DIR}/samples.npy', samples_new)

if not 'pid' in processes.dtype.names:
    #processes_new = np.array([pids, *(processes[col] for col in processes.dtype.names)], dtype=dtype_process)
    processes_new = np.empty(len(processes), dtype=dtype_process)
    processes_new['pid'] = np.arange(len(processes))
    for col in processes.dtype.names:
        processes_new[col] = processes[col]

    if len(processes) == len(processes_new):
        np.save(f'{INDEX_DIR}/processes.npy', processes_new)
        
print("Conversion successful")

Conversion successful


Testing

In [43]:
rf = ur.open('/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH/output/zhh_PreSelection_llHH.root')

In [55]:
tree = rf['eventTree']
rf_keys = tree.keys()

rf['eventTree'][rf_keys[0]].array()

<Array [403001] type='1 * int32'>

In [5]:
processes

array([('2f_z_bhabhang', '2f_z_bhabhang_LR', -1,  1, 1.33070797e+05, 7.84000015e+01, 250127),
       ('qqhh', 'qqhh_RL',  1, -1, 1.44522369e-01, 1.02380312e-04, 403010),
       ('e3e3hh', 'e3e3hh_LR', -1,  1, 1.11406688e-02, 5.84571035e-06, 403003),
       ('n23n23qqh', 'n23n23qqh_LR', -1,  1, 2.59607166e-01, 3.96785181e-04, 403017),
       ('2f_z_nung', '2f_z_nung_LR', -1,  1, 2.80930996e+04, 3.27000008e+01, 250130),
       ('n23n23qqh', 'n23n23qqh_RL',  1, -1, 1.10307105e-01, 1.66818631e-04, 403018),
       ('yyvllv', 'yyvllv_LR', -1,  1, 7.87087555e+01, 7.24000037e-02, 108665),
       ('4f_lowmee_sze_l', '4f_lowmee_sze_l_LR', -1,  1, 5.46110010e+00, 2.60000005e-02, 250119),
       ('yyveyx', 'yyveyx_LR', -1,  1, 1.17086838e+02, 8.83999988e-02, 108668),
       ('2f_z_h', '2f_z_h_LR', -1,  1, 3.24705000e+04, 8.88000031e+01, 250114),
       ('xxxyyx', 'xxxyyx_LR', -1,  1, 1.57004671e+01, 6.85999990e-02, 108687),
       ('yycyyc', 'yycyyc_RL',  1, -1, 6.06516876e+01, 2.98000002e+00, 108

In [6]:
samples

array([(250127, '2f_z_bhabhang', '2f_z_bhabhang_LR', 18600, -1,  1, '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_bhabhaNg/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250127.P2f_z_bhabhang.eL.pR.n005.d_dstm_10609_88.slcio'),
       (403010, 'qqhh', 'qqhh_RL',  5988,  1, -1, '/pnfs/desy.de/ilc/prod/ilc/mc-2020/ild/dst-merged/500-TDR_ws/hh/ILD_l5_o1_v02/v02-02-03/00015747/000/rv02-02-03.sv02-02-03.mILD_l5_o1_v02.E500-TDR_ws.I403010.Pqqhh.eR.pL.n012.d_dstm_15747_103.slcio'),
       (403003, 'e3e3hh', 'e3e3hh_LR',  8000, -1,  1, '/pnfs/desy.de/ilc/prod/ilc/mc-2020/ild/dst-merged/500-TDR_ws/hh/ILD_l5_o1_v02/v02-02-03/00015740/000/rv02-02-03.sv02-02-03.mILD_l5_o1_v02.E500-TDR_ws.I403003.Pe3e3hh.eL.pR.n001.d_dstm_15740_16.slcio'),
       ...,
       (250034, '4f_sze_l', '4f_sze_l_LR', 19600, -1,  1, '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/4f_singleZee_leptonic/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-T

Prototyping

In [4]:
results = get_preselection_passes(DATA_ROOT, processes)

TypeError: get_preselection_passes() missing 1 required positional argument: 'processes'

In [42]:
for entry in results:
    print(f'{entry["id"]} | {entry["n_gen"]} events | wt: {entry["weight"]} | {entry["n_pass_llhh"]} : {entry["n_pass_vvhh"]} : {entry["n_pass_qqhh"]}')
    
np.save(f'{REPO_ROOT}/prelim.npy', results)

2f_z_bhabhag_LL | 114285 events | wt: 32.65603256225586 | 475 : 0 : 2
2f_z_bhabhag_LR | 170071 events | wt: 50.15360641479492 | 663 : 0 : 0
2f_z_bhabhag_RL | 94916 events | wt: 4.973808288574219 | 378 : 0 : 0
2f_z_bhabhag_RR | 170610 events | wt: 4.506178855895996 | 682 : 0 : 1
2f_z_bhabhang_LL | 2237263 events | wt: 69.78526306152344 | 2298 : 0 : 7
2f_z_bhabhang_LR | 2460179 events | wt: 126.57032775878906 | 2844 : 0 : 7
2f_z_bhabhang_RL | 2553707 events | wt: 7.139760971069336 | 2801 : 0 : 3
2f_z_bhabhang_RR | 2404642 events | wt: 13.398371696472168 | 2537 : 0 : 3
2f_z_h_LR | 823873 events | wt: 92.22412872314453 | 55 : 278 : 1940
2f_z_h_RL | 550085 events | wt: 4.579761505126953 | 30 : 156 : 1229
2f_z_l_LR | 292163 events | wt: 44.43183135986328 | 134 : 6 : 14
2f_z_l_RL | 229318 events | wt: 2.6525626182556152 | 94 : 4 : 6
2f_z_nung_LR | 13688998 events | wt: 4.802239894866943 | 2 : 0 : 0
2f_z_nung_RL | 964568 events | wt: 0.2812284827232361 | 0 : 0 : 0
4f_lowmee_sze_l_LL | 196 even